In [1]:
# Parameters
cpu = 8
group_name = "DG_Glut"
mem_gb = 10


In [2]:
import pandas as pd
import subprocess
import numpy as np
import pyBigWig
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
#group_name = 'DG_Glut'

In [4]:
home_dir = '/home/qzeng_salk_edu/project/241018_pre_ml'
ct = group_name
dar_dir = '/data/combined_DARs_redo'
abc_dir = '/data/celltype_age_RPM_files'
_type = 'up_peaks'

In [5]:
dar_ct_dict = {'L23_IT_CTX_Glut':'L2-3_IT_CTX_Glut'}
bw_ct_dict = {'L23_IT_CTX_Glut':'L2_3_IT_CTX_Glut'}
if ct in bw_ct_dict.keys():
    dar_ct = dar_ct_dict[ct]
    bw_ct = bw_ct_dict[ct]
else:
    dar_ct = ct
    bw_ct = ct

In [6]:
atac_df  = pd.read_csv(f'{dar_dir}/diff_peaks_{dar_ct}_2vs18.csv', sep=',', index_col=0)

In [7]:
# add a chrom, start, end based on the index 
atac_df['chrom'] = atac_df.index.str.split(':').str[0]
atac_df['start'] = atac_df.index.str.split(':').str[1].str.split('-').str[0].astype(int)
atac_df['end'] = atac_df.index.str.split(':').str[1].str.split('-').str[1].astype(int)
atac_df

,log2(fold_change),p-value,adjusted p-value,chrom,start,end
feature name,,,,,,
chr13:3343561-3344062,2.850578,0.0,0.0,chr13,3343561,3344062
chr13:68173417-68173918,4.809919,0.0,0.0,chr13,68173417,68173918
chr13:68258227-68258728,6.678465,0.0,0.0,chr13,68258227,68258728
chr4:122566918-122567419,5.096911,0.0,0.0,chr4,122566918,122567419
chr4:144455014-144455515,6.832115,0.0,0.0,chr4,144455014,144455515
...,...,...,...,...,...,...
chrX:131567988-131568489,-0.361925,1.0,1.0,chrX,131567988,131568489
chrX:132897971-132898472,-0.301542,1.0,1.0,chrX,132897971,132898472
chrX:147543922-147544423,-0.250100,1.0,1.0,chrX,147543922,147544423


In [8]:
up_atac_df = pd.read_csv(f'{dar_dir}/up_peaks_{dar_ct}.bed', sep='\t', header = None)
#add index like in the atac_df
up_atac_df.index = up_atac_df[0] + ':' + up_atac_df[1].astype(str) + '-' + up_atac_df[2].astype(str)
# add the log(fold_change) baes on the info in atac_df
up_atac_df['log2FoldChange'] = atac_df['log2(fold_change)']
up_atac_df.head()

,0,1,2,log2FoldChange
chr13:3343561-3344062,chr13,3343561,3344062,2.850578
chr13:68173417-68173918,chr13,68173417,68173918,4.809919
chr13:68258227-68258728,chr13,68258227,68258728,6.678465
chr4:122566918-122567419,chr4,122566918,122567419,5.096911
chr4:144455014-144455515,chr4,144455014,144455515,6.832115


In [9]:
# write the same for down_peaks
down_atac_df = pd.read_csv(f'{dar_dir}/down_peaks_{dar_ct}.bed', sep='\t', header = None)
down_atac_df.index = down_atac_df[0] + ':' + down_atac_df[1].astype(str) + '-' + down_atac_df[2].astype(str)
down_atac_df['log2FoldChange'] = atac_df['log2(fold_change)']
down_atac_df.head()

,0,1,2,log2FoldChange
chr9:123461806-123462307,chr9,123461806,123462307,-0.885548
chr6:55680898-55681399,chr6,55680898,55681399,-3.092511
chr3:96557574-96558075,chr3,96557574,96558075,-1.550956
chr2:116045011-116045512,chr2,116045011,116045512,-2.121747
chr5:17574375-17574876,chr5,17574375,17574876,-1.345688


In [10]:
# combina the down and up
atac_df = pd.concat([up_atac_df, down_atac_df])
# save atac_df to bed
atac_df.to_csv(f'{ct}.aDAR.bed', sep='\t', header=False, index=False)

In [11]:
# use bedtools intersect to find the genes that overlap with the DMRs
command = f"bedtools intersect -a {ct}.aDAR.bed -b {home_dir}/mm10_gene_2kb.bed -wa -wb > {ct}.aDAR_gene.bed"
subprocess.run(command, shell=True)

CompletedProcess(args='bedtools intersect -a DG_Glut.aDAR.bed -b /home/qzeng_salk_edu/project/241018_pre_ml/mm10_gene_2kb.bed -wa -wb > DG_Glut.aDAR_gene.bed', returncode=0)

In [12]:
subprocess.run(f"rm {ct}.aDAR.bed", shell=True)

CompletedProcess(args='rm DG_Glut.aDAR.bed', returncode=0)

In [13]:
dar_gene_df = pd.read_csv(f'{ct}.aDAR_gene.bed', sep='\t', header=None)
dar_gene_df.columns = ['chr','start','end','log2(old/young)','gene_chr','gene_start','gene_end','gene_id','gene_strand','gene_name','gene_type']
dar_gene_df.head()

,chr,start,end,log2(old/young),gene_chr,gene_start,gene_end,gene_id,gene_strand,gene_name,gene_type
0,chr4,144455014,144455515,6.832115,chr4,144451009,144455317,ENSMUSG00000095213.1,-,Gm9944,protein_coding
1,chr4,144455014,144455515,6.832115,chr4,144451771,144465756,ENSMUSG00000078507.1,-,Aadacl3,protein_coding
2,chr4,144465321,144465822,6.667610,chr4,144451771,144465756,ENSMUSG00000078507.1,-,Aadacl3,protein_coding
3,chr7,15749286,15749787,6.467640,chr7,15748131,15761800,ENSMUSG00000074366.5,+,Obox5,protein_coding
4,chr17,13546492,13546993,1.172727,chr17,13480553,13763825,ENSMUSG00000038347.10,-,Tcte2,protein_coding


In [14]:
atac_rmp = pd.read_csv(f"{abc_dir}/{ct}_RPM.txt", sep = '\t')
# add three coolumns chr, start and end based on ethe index 
atac_rmp['chr'] = atac_rmp.index.str.split(':').str[0]
atac_rmp['start'] = atac_rmp.index.str.split(':').str[1].str.split('-').str[0].astype(int)
atac_rmp['end'] = atac_rmp.index.str.split(':').str[1].str.split('-').str[1].astype(int)
atac_rmp

,DG Glut:2mo,DG Glut:9mo,DG Glut:18mo,chr,start,end
chr1:3012480-3012981,2.535191,2.240782,2.096444,chr1,3012480,3012981
chr1:3060761-3061262,1.626886,1.176787,1.150766,chr1,3060761,3061262
chr1:3113502-3114003,5.274336,4.421410,4.154910,chr1,3113502,3114003
chr1:3112919-3113420,2.165229,1.654269,1.648291,chr1,3112919,3113420
chr1:3120282-3120783,2.359696,2.229503,2.111636,chr1,3120282,3120783
...,...,...,...,...,...,...
chrY:90807492-90807993,8.243522,10.700112,9.339052,chrY,90807492,90807993
chrY:90806986-90807487,2.630053,3.086715,2.255957,chrY,90806986,90807487
chrY:90808570-90809071,13.942366,21.106967,20.147894,chrY,90808570,90809071
chrY:90810721-90811222,2.589737,3.834896,3.023134,chrY,90810721,90811222


In [15]:
dar_gene_df.index = dar_gene_df['chr'] + ':' + dar_gene_df['start'].astype(str) + '-' + dar_gene_df['end'].astype(str)

In [16]:
shared_index = dar_gene_df.index.intersection(atac_rmp.index)
len(shared_index)

16617

In [17]:
dar_gene_df['2mo'] = atac_rmp.iloc[:,0].to_dict()
dar_gene_df['9mo'] = atac_rmp.iloc[:,1].to_dict()
dar_gene_df['18mo'] = atac_rmp.iloc[:,2].to_dict()

In [18]:
non_na_dar_gene_df = dar_gene_df[~dar_gene_df.isna().any(axis=1)]
with_na_dar_gene_df =  dar_gene_df[dar_gene_df.isna().any(axis=1)]

In [19]:
# use PyBigwig to get the atac information for the bw files
bw_dir = "/data/from_luisa/female_atac"
age_list = ['2mo', '9mo', '18mo']
total_counts_df = {}
for age in age_list:
    bw = pyBigWig.open(f"{bw_dir}/Female_{bw_ct}_{age}_merge.bw")
    total_counts = []
    for dmr_id in tqdm(with_na_dar_gene_df.index):
        _chr, start, end = dmr_id.split(':')[0], dmr_id.split(':')[1].split('-')[0], dmr_id.split(':')[1].split('-')[1]
        start, end = int(start), int(end)
        _count = bw.stats(_chr, start, end, type="mean")
        total_counts.append(_count[0])
        
    total_counts_df[age] = total_counts
total_counts_df = pd.DataFrame(total_counts_df)
    

  0%|          | 0/983 [00:00<?, ?it/s]

  0%|          | 1/983 [00:00<02:01,  8.11it/s]

  0%|          | 2/983 [00:00<02:49,  5.79it/s]

  0%|          | 3/983 [00:00<02:48,  5.83it/s]

  0%|          | 4/983 [00:00<02:21,  6.94it/s]

...

 96%|█████████▋| 948/983 [00:36<00:00, 35.30it/s]

 98%|█████████▊| 961/983 [00:36<00:00, 49.93it/s]

 98%|█████████▊| 967/983 [00:36<00:00, 47.79it/s]

 99%|█████████▉| 975/983 [00:36<00:00, 42.12it/s]

100%|██████████| 983/983 [00:36<00:00, 26.71it/s]

In [20]:
for age in age_list:
    with_na_dar_gene_df[age] = total_counts_df[age].values

/var/tmp/ipykernel_77952/436241227.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  with_na_dar_gene_df[age] = total_counts_df[age].values


In [21]:
with_na_dar_gene_df['log2(18mo/2mo)'] = np.log2(with_na_dar_gene_df['18mo']/with_na_dar_gene_df['2mo'])

/var/tmp/ipykernel_77952/2204217185.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  with_na_dar_gene_df['log2(18mo/2mo)'] = np.log2(with_na_dar_gene_df['18mo']/with_na_dar_gene_df['2mo'])


In [22]:
# map rows in dar_gene_df that is in with_na_dar_gene_df.index, give 2mo column values based on with_na_dar_gene_df['2mo'].to_dict()
dar_gene_df.loc[with_na_dar_gene_df.index, '2mo'] = with_na_dar_gene_df['2mo'].to_dict()
dar_gene_df.loc[with_na_dar_gene_df.index, '9mo'] = with_na_dar_gene_df['9mo'].to_dict()
dar_gene_df.loc[with_na_dar_gene_df.index, '18mo'] = with_na_dar_gene_df['18mo'].to_dict()

In [23]:
dar_gene_df.to_csv(f'{home_dir}/ml_input/{ct}/{ct}.aDAR_gene.csv',  index=False)

In [24]:
subprocess.run(f"rm {ct}.aDAR_gene.bed", shell=True)

CompletedProcess(args='rm DG_Glut.aDAR_gene.bed', returncode=0)